
# Packing CAT Cell demo notebook
The main functions of this demo are located in [chordal_axis_transform.py](../irregular_object_packing/packing/chordal_axis_transform.py)


### Contents
1. Simple CAT Cell demo
2. Larger Demo with cylindrical container and multiple blood cell meshes.

### Pre-requisites
Please install the required packages using the following command:

```bash
pip install -r requirements.txt
```




## Basic CAT Cell demo
The cell below shows a simple demo of computing cat faces. The container is a cube and the object os a cuboid rotated 45 degrees over the z axis. The red faces are the CAT faces that belong to the cube in the back. The yellow arrows are the normal vectors, which point inwards, of each face. the yellow faces are the cat faces that belong to the other objects.

In [20]:
import numpy as np
import pyvista as pv
def check_input_and_tetmesh_consistency(tetmesh, meshes):
    # assert tetmesh.n_points == sum([mesh.n_points for mesh in meshes]), "number of points in tetmesh and meshes do not match
    start_idx = 0
    for i, mesh in enumerate(meshes):
        end_idx = start_idx + mesh.n_points
        try:
            np.testing.assert_array_equal(tetmesh.points[start_idx:end_idx], mesh.points)
        except AssertionError as e:
            print(f"mesh {i} does not match tetmesh {e}")
        start_idx = end_idx


def compute_steiner_points(tetmesh, meshes):
    steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
    print(f"N steiner points: {len(steiner_points)}")
    return steiner_points

def print_cat_analysis(cat_meshes):
    for i, cat in enumerate(cat_meshes):
        print(f"cat {i}: {cat.n_points}, is manifold: {cat.is_manifold}")


## 4 Cubes

In [21]:
from irregular_object_packing.cat.chordal_axis_transform import compute_cdt, compute_cat_faces, filter_relevant_cells


from irregular_object_packing.cat.utils import get_tetmesh_cell_arrays

container = pv.Cube(center=(0, 0, 0), x_length=4, y_length=4, z_length=3).triangulate().clean()

cube4 = pv.Cube(center=(-1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube1 = pv.Cube(center=(1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube2 = pv.Cube(center=(1, -1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube3 = pv.Cube(center=(-1, 1, 0), x_length=1, y_length=1, z_length=1).triangulate().clean()
cube5 = pv.Cube(center=(0, 0, 0), x_length=0.9, y_length=2, z_length=1).triangulate().clean()

meshes = [cube1, cube2, cube3, cube4, container]#, cube5]
tetmesh = compute_cdt(meshes)

tet_cells = get_tetmesh_cell_arrays(tetmesh)
npoints_for_meshes = [mesh.n_points for mesh in meshes]
npoints_for_meshes[-1] += compute_steiner_points(tetmesh, meshes).shape[0]
relevant_cells, filtered_out = filter_relevant_cells(tet_cells, npoints_for_meshes)
assert len(filtered_out) != 0


plotter = pv.Plotter()
for mesh in meshes:
    plotter.add_mesh(mesh, color="red", opacity=0.5, show_edges=True)
plotter.add_points(tetmesh.points, color="black", opacity=1, show_vertices=True)
# plotter.add_mesh_clip_plane(tet_points, color="white", opacity=1, show_edges=True)
# plotter.add_mesh_clip_plane(relevant_cells, color="white", opacity=1, show_edges=True, crinkle=False)
plotter.show()


Delaunizing vertices...
N steiner points: 0
Delaunay seconds:  0.000303

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  1.2e-05
Total running seconds:  0.000377

Statistics:

  Input points: 40

  Mesh points: 40
  Mesh tetrahedra: 154
  Mesh faces: 314
  Mesh edges: 199
  Convex hull faces: 12



Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x1036196c0_13&reconnect=auto' style='width:…

In [22]:

from irregular_object_packing.mesh.utils import convert_faces_to_polydata_input

steiner_points = compute_steiner_points(tetmesh, meshes)
n_points_per_obj = [len(mesh.points) for mesh in meshes] + [len(container.points)+ len(steiner_points)]
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])

cat_meshes = []
for cat_cell in cat_cells[:-1]:
    cat_meshes.append(pv.PolyData(*convert_faces_to_polydata_input(cat_cell)))
    

print_cat_analysis(cat_meshes)
check_input_and_tetmesh_consistency(tetmesh, meshes+ [container])

plotter = pv.Plotter()
# plotter.add_mesh_clip_plane(tetmesh, color="grey", show_edges=True, opacity=0.9, crinkle=True)
for mesh in meshes:
    plotter.add_mesh(mesh, color="red", show_edges=True, opacity=1,)

plotter.show()



N steiner points: 0
cat 0: 78, is manifold: True
cat 1: 58, is manifold: True
cat 2: 58, is manifold: True
cat 3: 62, is manifold: True
cat 4: 108, is manifold: True
mesh 5 does not match tetmesh 
Arrays are not equal

(shapes (0, 3), (8, 3) mismatch)
 x: pyvista_ndarray([], shape=(0, 3), dtype=float64)
 y: pyvista_ndarray([[-2. , -2. , -1.5],
                 [-2. , -2. ,  1.5],
                 [-2. ,  2. , -1.5],...


Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x2973d97e0_14&reconnect=auto' style='width:…

In [23]:
from irregular_object_packing.mesh.collision import compute_collision


cell_id = 3
def plot_single_mesh_with_cat(cell_id):
    plotter = pv.Plotter()
    plotter.add_mesh(meshes[cell_id], color="red", show_edges=True, opacity=1,)
    plotter.add_mesh(cat_meshes[cell_id].explode(), color="yellow", show_edges=True, opacity=0.8,)
    plotter.show()

plot_single_mesh_with_cat(cell_id)

Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x2940a4400_15&reconnect=auto' style='width:…

## Sphere inside a sphere

In [24]:

container = pv.Sphere(radius=2).triangulate().clean().rotate_y(1)
sphere = pv.Sphere(radius=1).triangulate().clean()
CDT_DEFAULTS = {
    "nobisect": True,
    "steinerleft": 0,
    "minratio": 10.0,
    "quality": False,
    "cdt": True,
    "switches": "O0/0",
}
meshes = [sphere, container]#, cube5]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
# assert(len(tetmesh.points) == len(sphere.points) + len(container.points))
# assert(len(tetmesh.points) == tetmesh.n_points)

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]

# TODO: Filter out steinerpoints. Probably the last points.
print(f'number of steiner points: {len(steiner_points)}')

Delaunizing vertices...
Delaunay seconds:  0.007352

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000164
Total running seconds:  0.007599

Statistics:

  Input points: 1684

  Mesh points: 1684
  Mesh tetrahedra: 8915
  Mesh faces: 18670
  Mesh edges: 11438
  Convex hull faces: 1680

number of steiner points: 0


In [25]:
n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
# FIXME: There are only 2 objects in this situation, however, there are object ids 0,1 and 2
print_cat_analysis(cat_meshes)

cat 0: 3477, is manifold: True
cat 1: 3477, is manifold: True


In [26]:
plotter = pv.Plotter()
plotter.add_mesh(sphere, color="red", show_edges=True, opacity=0.8)
plotter.add_mesh(container, color="white", opacity=0.8)
plotter.add_mesh(cat_meshes[0], color="yellow", show_edges=True, opacity=0.8)
if len(steiner_points) != 0:
    plotter.add_points(steiner_points, color="purple", render_points_as_spheres=True, point_size=10)
plotter.show()


Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x2973d8520_16&reconnect=auto' style='width:…

In [27]:
container = pv.Sphere(radius=2).triangulate().clean().rotate_y(1)
sphere_left = pv.Sphere(radius=0.5, center=(-0.6,0,0)).triangulate().clean()
sphere_right = pv.Sphere(radius=0.5, center=(0.6,0,0)).triangulate().clean()
meshes = [sphere_left, sphere_right, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)


steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))

n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)


Delaunizing vertices...
Delaunay seconds:  0.014009

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000219
Total running seconds:  0.014429

Statistics:

  Input points: 2526

  Mesh points: 2526
  Mesh tetrahedra: 14806
  Mesh faces: 30452
  Mesh edges: 18171
  Convex hull faces: 1680

steiner points:  0
cat 0: 3442, is manifold: True
cat 1: 3442, is manifold: True
cat 2: 5108, is manifold: True


In [28]:
plotter= pv.Plotter()
def plot_cat_cells(plotter):
    for mesh in meshes[:-1]:
        plotter.add_mesh(mesh, color="red", show_edges=True, opacity=0.9)
    for mesh in cat_meshes[:-1]:
        plotter.add_mesh(mesh, color="yellow", show_edges=True, opacity=0.7)

    plotter.add_mesh(container, color="white", opacity=0.6)

    if len(steiner_points) != 0:
        plotter.add_points(steiner_points, color="purple", render_points_as_spheres=True, point_size=20)
    plotter.show()

plot_cat_cells(plotter)

plotter= pv.Plotter()
plotter.add_mesh(container, color="white", opacity=0.8)
plotter.add_mesh(cat_meshes[-1], color="green", show_edges=True, opacity=0.8)
plotter.show()

Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x298a42920_17&reconnect=auto' style='width:…

Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x16e1c6470_18&reconnect=auto' style='width:…

In [29]:
sphere1 = pv.Sphere(radius=0.2, center=(-0.6,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere2 = pv.Sphere(radius=0.2, center=(0.6,0,0) , theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere3 = pv.Sphere(radius=0.2, center=(0,0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere4 = pv.Sphere(radius=0.2, center=(0,-0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere5 = pv.Sphere(radius=0.2, center=(0,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()

meshes = [sphere1, sphere2, sphere3, sphere4, sphere5, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))


n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)
check_input_and_tetmesh_consistency(tetmesh, meshes)

Delaunizing vertices...
steiner points:  0
Delaunay seconds:  0.007505

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000107
Total running seconds:  0.007669

Statistics:

  Input points: 1252

  Mesh points: 1252
  Mesh tetrahedra: 6055
  Mesh faces: 12950
  Mesh edges: 8146
  Convex hull faces: 1680

cat 0: 889, is manifold: True
cat 1: 889, is manifold: True
cat 2: 889, is manifold: True
cat 3: 890, is manifold: True
cat 4: 880, is manifold: True
cat 5: 2517, is manifold: True


In [30]:
plot_cat_cells(pv.Plotter())

plotter = pv.Plotter()
plotter.add_mesh(cat_meshes[-1], color="red", show_edges=True, opacity=0.8)
plotter.show()



Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x136f3b130_19&reconnect=auto' style='width:…

Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x294a8ad70_20&reconnect=auto' style='width:…

In [31]:
container = pv.Sphere(radius=1, theta_resolution=10, phi_resolution=10).triangulate().clean().rotate_y(1)
sphere1 = pv.Sphere(radius=0.2, center=(-0.6,0,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere2 = pv.Sphere(radius=0.2, center=(0.6,0,0) , theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere3 = pv.Sphere(radius=0.2, center=(0,0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
sphere4 = pv.Sphere(radius=0.2, center=(0,-0.6,0), theta_resolution=10, phi_resolution=10).triangulate().clean()
cube_5 = pv.Cube(center=(0,0,0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()
cube_6 = pv.Cube(center=(0,0,-0.6), x_length=0.2, y_length=0.2, z_length=0.2).triangulate().clean()


meshes = [sphere1, sphere2, sphere3, sphere4, cube_5, cube_6, container]
tetmesh = compute_cdt(meshes, CDT_DEFAULTS)

steiner_points = tetmesh.points[range(tetmesh.n_points - sum([mesh.n_points for mesh in meshes]))]
print("steiner points: ", len(steiner_points))

n_points_per_obj = [mesh.n_points for mesh in meshes]
n_points_per_obj[-1] += len(steiner_points)
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]
print_cat_analysis(cat_meshes)

steiner points: Delaunizing vertices...
 0
Delaunay seconds:  0.003756

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  3.5e-05
Total running seconds:  0.003833

Statistics:

  Input points: 426

  Mesh points: 426
  Mesh tetrahedra: 2399
  Mesh faces: 4878
  Mesh edges: 2904
  Convex hull faces: 160

cat 0: 397, is manifold: True
cat 1: 399, is manifold: True
cat 2: 385, is manifold: True
cat 3: 386, is manifold: True
cat 4: 214, is manifold: True
cat 5: 214, is manifold: True
cat 6: 851, is manifold: True


In [32]:
plot_cat_cells(pv.Plotter())

Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x16e032d40_21&reconnect=auto' style='width:…

In [35]:
from irregular_object_packing.mesh.transform import scale_and_center_mesh


container = container

rb_cell = pv.read("./../data/mesh/RBC_normal.stl")
rb_cell = scale_and_center_mesh(rb_cell, sphere1.volume)
cell1 = rb_cell.copy().translate((-0.6,0,0))
cell2 = rb_cell.copy().translate((0.6,0,0))
cell3 = rb_cell.copy().translate((0,0.6,0))
cell4 = rb_cell.copy().translate((0,-0.6,0))
cell5 = rb_cell.copy().translate((0,0,0.6))
cell6 = rb_cell.copy().translate((0,0,-0.6))

meshes = [cell1, cell2, cell3, cell4, cell5, cell6, container]

tetmesh = compute_cdt(meshes, CDT_DEFAULTS)
n_points_per_obj = [mesh.n_points for mesh in meshes]
face_normals, cat_cells, face_normals_pp = compute_cat_faces(tetmesh, n_points_per_obj, [])
cat_meshes = [pv.PolyData(*convert_faces_to_polydata_input(cell)) for cell in cat_cells]

Delaunizing vertices...
Delaunay seconds:  0.025706

Writing nodes.
Writing elements.
Writing faces.

Output seconds:  0.000286
Total running seconds:  0.02616

Statistics:

  Input points: 3934

  Mesh points: 3934
  Mesh tetrahedra: 26932
  Mesh faces: 53944
  Mesh edges: 30945
  Convex hull faces: 160



In [36]:
plot_cat_cells(pv.Plotter())

Widget(value="<iframe src='http://localhost:50283/index.html?ui=P_0x2cce6c700_22&reconnect=auto' style='width:…